### Using Tensorflow for object detection and extraction

#### We will implementing transfer learning with ImageAI for custom object detection

In [2]:
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

# Imports

In [5]:
from imageai.Detection import ObjectDetection

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import xml.etree.ElementTree as xm

from keras_tqdm import TQDMNotebookCallback

import tensorflow as tf

from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Lambda, Input, Reshape
from tensorflow.keras.callbacks import TensorBoard

from time import strftime

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from PIL import Image

import matplotlib.pyplot as plt

%matplotlib inline

# Constants

In [3]:
shelf_images = "./GroceryDataset/images/ShelfImages/"
product_images = "./GroceryDataset/images/ProductImagesFromShelves/"


# Trying out the raw model

In [3]:
execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.1.0.h5"))
detector.loadModel()

In [7]:
os.getcwd()

'C:\\Users\\Nikhil\\Desktop\\Hackathon\\Intelligent-Conuter-Share-Analyzer\\ML Models'

In [10]:
os.path.join(execution_path , "resnet50_coco_best_v2.1.0.h5")

'C:\\Users\\Nikhil\\Desktop\\Hackathon\\Intelligent-Conuter-Share-Analyzer\\ML Models\\resnet50_coco_best_v2.1.0.h5'

In [19]:
os.path.join(execution_path , "rect1.png")

'C:\\Users\\Nikhil\\Desktop\\Hackathon\\Intelligent-Conuter-Share-Analyzer\\ML Models\\rect1.png'

In [4]:
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "Production Assets/Shelf.jpg"), output_image_path=os.path.join(execution_path , "imagenew.jpg"))

for eachObject in detections:
    print(eachObject["name"] , " : " , eachObject["percentage_probability"] )

bottle  :  61.22475266456604
vase  :  56.356436014175415


In [5]:
# import tarfile

# open_tarfile=tarfile.open("GroceryDataset_part1.tar.gz")
# open_tarfile.extractall(path='GroceryDataset_part1')
# open_tarfile.close()

In [6]:
# open_tarfile=tarfile.open("GroceryDataset_part2.tar.gz")
# open_tarfile.extractall(path='GroceryDataset_part2')
# open_tarfile.close()

# Formating the data set in required format

In [12]:
jpg_files = [ f for f in os.listdir(f'{shelf_images}') if f.endswith('JPG') ]
photos_df = pd.DataFrame([[f, f[:6], f[7:14]] for f in jpg_files], columns=['file', 'shelf_id', 'planogram_id'])

In [13]:
print(len(jpg_files))
print(jpg_files[:5])
photos_df.head(5)

354
['C1_P01_N1_S2_1.JPG', 'C1_P01_N1_S2_2.JPG', 'C1_P01_N1_S3_1.JPG', 'C1_P01_N1_S3_2.JPG', 'C1_P01_N1_S5_1.JPG']


,file,shelf_id,planogram_id
0,C1_P01_N1_S2_1.JPG,C1_P01,N1_S2_1
1,C1_P01_N1_S2_2.JPG,C1_P01,N1_S2_2
2,C1_P01_N1_S3_1.JPG,C1_P01,N1_S3_1
3,C1_P01_N1_S3_2.JPG,C1_P01,N1_S3_2
4,C1_P01_N1_S5_1.JPG,C1_P01,N1_S5_1


In [14]:
products_df = pd.DataFrame([[f[:18], f[:6], f[7:14], i, *map(int, f[19:-4].split('_'))]
                           for i in range(11)
                           for f in os.listdir(f'{product_images}{i}') if f.endswith('png')],
                          columns = ['file', 'shelf_id', 'planogram_id', 'category', 'xmin', 'ymin', 'w', 'h'])

# convert from width, height to xmax, ymax

products_df['xmax'] = products_df['xmin'] + products_df['w']
products_df['ymax'] = products_df['ymin'] + products_df['h']

In [15]:
print(products_df.shape)
(products_df.head(5))

(13184, 10)


,file,shelf_id,planogram_id,category,xmin,ymin,w,h,xmax,ymax
0,C1_P01_N1_S2_1.JPG,C1_P01,N1_S2_1,0,1008,1552,252,376,1260,1928
1,C1_P01_N1_S2_1.JPG,C1_P01,N1_S2_1,0,1028,928,252,376,1280,1304
2,C1_P01_N1_S2_1.JPG,C1_P01,N1_S2_1,0,24,872,244,392,268,1264
3,C1_P01_N1_S2_1.JPG,C1_P01,N1_S2_1,0,280,1568,252,376,532,1944
4,C1_P01_N1_S2_1.JPG,C1_P01,N1_S2_1,0,292,872,252,376,544,1248


# converting in pascal VOC format

#### Done in Create Annotation file

# Train with yolov3

In [3]:
from imageai.Detection.Custom import DetectionModelTrainer


In [4]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [5]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
execution_path = os.getcwd()

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory=os.path.join(execution_path ,"GroceryDataset\production"))
trainer.setTrainConfig(object_names_array=["0","1","2","3","4","5","6","7","8","9","10"], batch_size=2, num_experiments=10, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.71
Anchor Boxes generated.
Detection configuration saved in  C:\Users\Nikhil\Desktop\Hackathon\Intelligent-Conuter-Share-Analyzer\ML Models\GroceryDataset\production\json\detection_config.json
Evaluating over 109 samples taken from C:\Users\Nikhil\Desktop\Hackathon\Intelligent-Conuter-Share-Analyzer\ML Models\GroceryDataset\production\validation
Training over 245 samples  given at C:\Users\Nikhil\Desktop\Hackathon\Intelligent-Conuter-Share-Analyzer\ML Models\GroceryDataset\production\train
Training on: 	['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
Training with Batch Size:  2
Number of Training Samples:  245
Number of Validation Samples:  109
Number of Experiments:  10
Training with transfer learning from pretrained Model
Epoch 1/10
 10/984 [..............................] - ETA: 12:02 - loss: 138.6395 - yolo_layer_3_loss: 13.5365 - yolo_layer_4_loss: 29.0519 - yolo_layer_5_loss: 84.4